# Analysis of Sweep Results for Main Paper

### Imports:

In [9]:
from analysis_utils import rerun_trials, load_checkpoints
import numpy as np
import torch
from torch import nn
import neurogym as ngym
import matplotlib.pyplot as plt
import matplotlib as mpl

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(f'<h4>{string}</h4>'))

def printbigmd(string):
    display(Markdown(f'<h2><center style="color:red">{string}</center></h2>'))

### Set the root directory of data to analyze. <strong>User should customize:</strong>

In [2]:
neurogym_root = '/home/ws3/Desktop/james/neurogym/examples/'
task = 'ContextDecisionMaking-v0'
root = neurogym_root + task + '/'
printbigmd(f'Data root file: {root}')

<h2><center style="color:red">Data root file: /home/ws3/Desktop/james/neurogym/examples/ContextDecisionMaking-v0/</center></h2>

In [3]:
checkpoints, GD_iteration = load_checkpoints(root)
printbigmd(f'{len(checkpoints)} Checkpoints, over {GD_iteration[-1]} GD iterations')

<h2><center style="color:red">100 Checkpoints, over 19800 GD iterations</center></h2>

## Analysis Begins...

#### Define the task inputs and targets. These will be consistently used for all reruns:

In [4]:
kwargs = {'dt': 100}
seq_len = 100
ntrials = 256
dataset = ngym.Dataset(task, env_kwargs=kwargs, batch_size=ntrials, seq_len=seq_len)

for i in range(1):
    inputs, labels = dataset()
    inputs, labels = inputs.swapaxes(0,1), labels.swapaxes(0,1)
    targets = nn.functional.one_hot(torch.from_numpy(labels)).numpy()

printmd(f'Input size [B, T, n_in]: {inputs.shape}, Target size [B, T, n_out]: {targets.shape}')

/home/ws3/anaconda3/envs/james/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:69: UserWarning: WARN: Agent's minimum action space value is -infinity. This is probably too low.
  logger.warn(
/home/ws3/anaconda3/envs/james/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:73: UserWarning: WARN: Agent's maximum action space value is infinity. This is probably too high
  logger.warn(
/home/ws3/anaconda3/envs/james/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


<h4>Input size [B, T, n_in]: (256, 100, 7), Target size [B, T, n_out]: (256, 100, 3)</h4>

#### Rerun all checkpoint models on these data:

In [8]:
zs_all, adjs_all, losses_all = rerun_trials(inputs, targets, checkpoints, compute_adj = True)
printmd(f'zs shape is [checkpoints, trials, timesteps, n_hidden]: {zs_all.shape}')

Re-evaluating on the Same Data.


/home/ws3/Desktop/james/neurogym/examples/ContextDecisionMaking-v0/checkpoints/checkpoint_19800.pt: 100%|██████████| 100/100 [00:07<00:00, 13.26it/s]


<h4>zs shape is [checkpoints, trials, timesteps, n_hidden]: (100, 256, 100, 128)</h4>